In [8]:
import pandas as pd
import numpy as np

# Class which defines the Neural Network
class NeuralNetwork(object):
    def __init__(self,input_nodes,hidden_layer1_nodes,hidden_layer2_nodes,output_nodes):
        #Parameters
        self.inputsize = input_nodes
        self.hiddenlayer1size = hidden_layer1_nodes
        self.hiddenlayer2size = hidden_layer2_nodes
        self.outputsize = output_nodes
        
        #Weights
        self.W1 = np.random.randn(self.inputsize, self.hiddenlayer1size)
        self.W2 = np.random.randn(self.hiddenlayer1size, self.hiddenlayer2size)
        self.W3 = np.random.randn(self.hiddenlayer2size, self.outputsize)
        
        
    # The forward feed neural network
    def feedforward(self,x):
        #multiplying values from input neurons with layer 1 weights
        self.z_input_to_hidden_layer1 = np.dot(x,self.W1)
        self.z_hidden_layer1_output = self.relu(self.z_input_to_hidden_layer1)
        
        #multiplying values from hidden layer 1 neurons with layer 2 weights
        self.z_hidden_layer1_to_hidden_layer2 = np.dot(self.z_hidden_layer1_output,self.W2)
        self.z_hidden_layer2_output = self.relu(self.z_hidden_layer1_to_hidden_layer2)
        
        #multiplying values from hidden layer 2 neurons with layer 3 weights
        self.z_hidden_layer2_to_output = np.dot(self.z_hidden_layer2_output,self.W3)
        self.z_output = self.sigmoid(self.z_hidden_layer2_to_output)
        
        return self.z_output
    
    # The relu activation function    
    def relu(self,val):
        return np.maximum(val,0)
    
    # The sigmoid activation function    
    def sigmoid(self,val):
        return 1/(1+np.exp(-val))
    
    # The relu activation function
    def relu_derivative(self,val):
        return 1 * (val > 0)
    
    # Derivative of sigmoid function
    def sigmoid_derivative(self,val):
        return val * (1.0 - val)
    
    def backprop(self,o,y,x,learning_rate):
        
        self.output_error = y - o
        self.output_delta = self.output_error * self.sigmoid_derivative(o)
        
        for i in range(len(self.W3)):
            for j in range(len(self.W3[0])):
                self.W3[i][j] += (self.z_hidden_layer2_output[i] * self.output_delta[j] * learning_rate)
                
                
        self.hidden_layer2_error = np.dot(self.output_delta,self.W3.T)
        self.hidden_layer2_delta = self.hidden_layer2_error * self.relu_derivative(self.z_hidden_layer2_output)

        for i in range(len(self.W2)):
            for j in range(len(self.W2[0])):
                self.W2[i][j] += (self.z_hidden_layer1_output[i] * self.hidden_layer2_delta[j] * learning_rate)
                

        self.hidden_layer1_error = np.dot(self.hidden_layer2_delta,self.W2.T)
        self.hidden_layer1_delta = self.hidden_layer1_error * self.relu_derivative(self.z_hidden_layer1_output)


        for i in range(len(self.W1)):
            for j in range(len(self.W1[0])):
                self.W1[i][j] += (x[i] * self.hidden_layer1_delta[j] * learning_rate)
        
        
    def train(self,x,y,learning_rate):
        o = []
        mse = 0
        for i in range(x.shape[0]):
            o.append(self.feedforward(x[i]))
            mse = (y[i] - o[i]) ** 2
        for i in range(x.shape[0]):
            self.backprop(o[i],y[i],x[i],learning_rate)
        
        mse = mse * 0.5/len(x)
        print("Error = ","{:5.10f}".format(mse[0]))
        return mse
        
        
    def test(self,x):
        o = self.feedforward(x)
        return o
    
    
    def accuracy(self,o,y):
        size = y.shape[0]
        count = 0
        
        for i in range(y.shape[0]):
            if(o[i] == y[i]):
                count += 1
                
        return count/size

In [9]:
def split_into_kfolds(df,kfolds):
    df_list = []
    
    size = len(df)
    count = int(size/kfolds)

    for i in range(kfolds):
        beg_index = i * count
        end_index = (i+1) * count
        if i == kfolds-1:
            end_index = size

        df_list.append(df[beg_index:end_index])
        
    return df_list

def normalize(x):
    result = x.copy()
    for feature_name in x.columns:
        max_value = x[feature_name].max()
        min_value = x[feature_name].min()
        result[feature_name] = (x[feature_name] - min_value) / (max_value - min_value)
    return result

In [10]:
%matplotlib tk

from sklearn.model_selection import train_test_split
from math import pow
from datetime import datetime
import matplotlib.pyplot as plt 

startTime = datetime.now()

df = pd.read_csv('ILPD.csv')
df = normalize(df)

col = ['Age','Gender','TB','DB','Alkphos','Sgpt','Sgot','TP','ALB','A/G']
col_class = 'Selector'


kfolds = 5
threshold = 0.5
learning_rate = 8 * pow(10,-4)
epochs = 30

df_list = split_into_kfolds(df,kfolds)

input_nodes = len(col)
hidden_layer1_nodes = 10
hidden_layer2_nodes = 5
output_node = 1


NN = NeuralNetwork(input_nodes,hidden_layer1_nodes,hidden_layer2_nodes,output_node)

score_train = []
score_test = []

count_epochs = range(1,epochs+1)
mse = []

for i in range(kfolds):
    print("Training on cross-validation ",i+1,"\n" + "\n")
    
    mse = []
    
    df_train = pd.DataFrame()
    
    for j in range(0,i):
        df_train = pd.concat([df_train,df_list[j]], ignore_index=True)
        
    for j in range(i+1,kfolds):
        df_train = pd.concat([df_train,df_list[j]], ignore_index=True)
    df_test = df_list[i]
    
    X_train = df_train[col]
    y_train = df_train[col_class]
    X_test = df_test[col]
    y_test = df_test[col_class]

    X_train, y_train, X_test, y_test = np.array(X_train), np.array(y_train), np.array(X_test), np.array(y_test)
    
    for epoch in range(0,epochs):
        print("Epoch ", epoch + 1," : ",end=" ")
        mse.append(NN.train(X_train,y_train,learning_rate))

    print("\n")
    
    o = []
    for k in range(X_test.shape[0]):
        o.append(NN.feedforward(X_test)[k])

        
    for k in range(len(o)):
        if o[k] > threshold:
            o[k] = 1
        else:
            o[k] = 0

    test_acc = NN.accuracy(o,y_test)
    score_test.append(test_acc)

    
    o1 = []
    for k in range(X_train.shape[0]):
        o1.append(NN.feedforward(X_train)[k])
        
    
    for k in range(len(o1)):
        if o1[k] > threshold:
            o1[k] = 1
        else:
            o1[k] = 0

    train_acc = NN.accuracy(o1,y_train)
    score_train.append(train_acc)

plt.plot(count_epochs, mse, color='green', linestyle='solid', linewidth = 2, marker='o', markerfacecolor='blue', markersize=9) 
# setting x and y axis range 
plt.ylim(min(mse),max(mse)) 
plt.xlim(1,epochs) 
  
# naming the x axis 
plt.xlabel('Epoch number') 
# naming the y axis 
plt.ylabel('MSE') 
  
# giving a title to my graph 
plt.title('Neural Network') 
  
# function to show the plot 
plt.show() 
    
train ,test = train_test_split(df, test_size = 0.25)

X_test = test[col]
y_test = test[col_class]
X_test, y_test = np.array(X_test), np.array(y_test)

o = []
for k in range(X_test.shape[0]):
    o.append(NN.feedforward(X_test)[k])

for k in range(len(o)):
    if o[k] > threshold:
        o[k] = 1
    else:
        o[k] = 0
        
final_score = NN.accuracy(o,y_test)

score_train_mean = sum(score_train)/len(score_train)
print("Training accuracy is : ", "{:5.4f}".format(score_train_mean))

score_test_mean = sum(score_test)/len(score_test)
print("Testing accuracy average of all folds is : ", "{:5.4f}".format(score_test_mean))

print("Final testing accuracy is : ","{:5.4f}".format(final_score))


print("\n")
print("Execution time in seconds = ", datetime.now() - startTime)

Training on cross-validation  1 


Epoch  1  :  Error =  0.0001499974
Epoch  2  :  Error =  0.0001833227
Epoch  3  :  Error =  0.0001840261
Epoch  4  :  Error =  0.0001807125
Epoch  5  :  Error =  0.0001762438
Epoch  6  :  Error =  0.0001709351
Epoch  7  :  Error =  0.0001647484
Epoch  8  :  Error =  0.0001577291
Epoch  9  :  Error =  0.0001496172
Epoch  10  :  Error =  0.0001408549
Epoch  11  :  Error =  0.0001311122
Epoch  12  :  Error =  0.0001224618
Epoch  13  :  Error =  0.0001102319
Epoch  14  :  Error =  0.0001125284
Epoch  15  :  Error =  0.0000581470
Epoch  16  :  Error =  0.0001859289
Epoch  17  :  Error =  0.0000231935
Epoch  18  :  Error =  0.0001927266
Epoch  19  :  Error =  0.0001930975
Epoch  20  :  Error =  0.0001933181
Epoch  21  :  Error =  0.0001933942
Epoch  22  :  Error =  0.0001934134
Epoch  23  :  Error =  0.0001934177
Epoch  24  :  Error =  0.0001934184
Epoch  25  :  Error =  0.0001934183
Epoch  26  :  Error =  0.0001934179
Epoch  27  :  Error =  0.0001934176
Ep